# 🔌CONEXÃO
Estabelecer conexão com os bancos

In [ ]:
import psycopg2
from psycopg2.extras import DictCursor
import fdb
from dotenv import load_dotenv
import os

load_dotenv()

# Conexões
cnx_dest = fdb.connect(
    user=os.getenv("FDB_USER"),
    password=os.getenv("FDB_PASS"),
    host=os.getenv("FDB_HOST"),
    port=int(os.getenv("FDB_PORT")),
    database=os.getenv("FDB_PATH"),
    charset="WIN1252"
)
cur_dest = cnx_dest.cursor()


cnx_orig = psycopg2.connect(
    user=os.getenv("PG_USER"),
    password=os.getenv("PG_PASS"),
    host=os.getenv("PG_HOST"),
    database=os.getenv("PG_DB"),
    options="-c search_path={}".format(os.getenv("PG_SCHEMA"))
)
cnx_orig.autocommit = True
cur_orig = cnx_orig.cursor(cursor_factory=DictCursor)

def commit():
    cnx_dest.commit()

# 🛠️ FERRAMENTAS
Funções, variáveis cache, hashmaps

In [ ]:
global cadest, empresa, exercicio
cadest = {}
empresa = cur_dest.execute("SELECT empresa FROM cadcli").fetchone()[0]
exercicio = '2025' #cur_dest.execute("SELECT mexer FROM cadcli").fetchone()[0]

def limpa_tabela(tabelas):
    for tabela in tabelas:
        cur_dest.execute(f"DELETE FROM {tabela}")
    commit()

def cria_coluna(tabela, coluna):
    try:
        cur_dest.execute(f"ALTER TABLE {tabela} ADD {coluna} VARCHAR(255)")
    except fdb.DatabaseError as e:
        print(f"Erro ao criar coluna {coluna} na tabela {tabela}: {e}")
    else:
        commit()

dict_modalidades = {
    "CO":{"Licit": "MAT / SERV - CONCORRENCIA", 
          "Modlic": "CON4",
          "Codmod": 4},
    "TP":{"Licit": "MAT / SERV - TOMADA", 
          "Modlic": "TOM3",
          "Codmod": 3},
    "CC":{"Licit": "MAT / SERV - CONVITE", 
          "Modlic": "CS01",
          "Codmod": 7},
    "LE":{"Licit": "LEILÃO", 
          "Modlic": "LEIL",
          "Codmod": 6},
    "DL":{"Licit": "DISPENSA", 
          "Modlic": "DI01",
          "Codmod": 1},
    "IL":{"Licit": "INEXIGIBILIDADE", 
          "Modlic": "IN01",
          "Codmod": 5},
    "PR":{"Licit": "PREGÃO PRESENCIAL", 
          "Modlic": "PP01",
          "Codmod": 8},
    "01":{"Licit": "PREGÃO PRESENCIAL", 
          "Modlic": "PP01",
          "Codmod": 8},
    "02":{"Licit": "PREGÃO ELETRÔNICO", 
          "Modlic": "PE01",
          "Codmod": 9},
    "03":{"Licit": "DISPENSA", 
          "Modlic": "DI01",
          "Codmod": 1},
    "04":{"Licit": "DISPENSA", 
          "Modlic": "DI01",
          "Codmod": 1},
    "06":{"Licit": "DISPENSA", 
          "Modlic": "DI01",
          "Codmod": 1},
    "CE":{"Licit": "CONCORRÊNCIA ELETRÔNICA", 
          "Modlic": "CE01",
          "Codmod": 13},
    "IN":{"Licit": "INEXIGIBILIDADE", 
          "Modlic": "IN01",
          "Codmod": 5},
    "PE":{"Licit": "PREGÃO ELETRÔNICO", 
          "Modlic": "PE01",
          "Codmod": 9},
}

def cria_coluna(tabela, coluna):
    try:
        cur_dest.execute(f"ALTER TABLE {tabela} ADD {coluna} VARCHAR(255)")
    except fdb.DatabaseError as e:
        print(f"Erro ao criar coluna {coluna} na tabela {tabela}: {e}")
    else:
        commit()

def to_cp1252_safe(value, replace_with=''):
    """
    Converte uma string para cp1252 de forma segura.
    Remove ou substitui caracteres inválidos.

    :param value: valor a converter
    :param replace_with: string usada no lugar de caracteres inválidos (default = '')
    :return: valor limpo
    """
    if isinstance(value, str):
        try:
            # Tenta converter diretamente
            value.encode('cp1252')
            return value
        except UnicodeEncodeError:
            # Remove/substitui caracteres inválidos
            return value.encode('cp1252', errors='replace').decode('cp1252').replace('?', replace_with)
    return value

cur_dest.execute("SELECT codreduz, max(cadpro) FROM cadest group by 1")
if cur_dest.description is None:
    print("CADEST VAZIA!")
else:
    cadest = {k:v for k,v in cur_dest.execute("SELECT codreduz, max(cadpro) FROM cadest group by 1").fetchall()}


# 🛒 COMPRAS
<p>Extração, tratamento e carregamento dos dados referentes ao módulo compras</p>

In [ ]:
cur_dest.execute("""
execute block as
		begin
		DELETE FROM ICADREQ;
		DELETE FROM REQUI;
		DELETE FROM ICADPED;
		DELETE FROM CADPED;
		DELETE FROM regpreco;
		DELETE FROM regprecohis;
		DELETE FROM regprecodoc;
		DELETE FROM CADPRO_SALDO_ANT;
		DELETE FROM CADPROLIC_DETALHE_FIC;
		DELETE FROM CADPRO;
		DELETE FROM CADPRO_FINAL;
		DELETE FROM CADPRO_LANCE;
		DELETE FROM CADPRO_PROPOSTA;
		DELETE FROM PROLICS;
		DELETE FROM PROLIC;
		DELETE FROM CADPRO_STATUS;
		DELETE FROM CADLIC_SESSAO;
		DELETE FROM CADPROLIC_DETALHE;
		DELETE FROM CADPROLIC;
		DELETE FROM CADLIC;
		DELETE FROM VCADORC;
		DELETE FROM FCADORC;
		DELETE FROM ICADORC;
		DELETE FROM CADORC;
		DELETE FROM CADEST;
		DELETE FROM CENTROCUSTO;
		DELETE FROM DESTINO;
		DELETE FROM DESFORCRC_PADRAO;
		end;
""")
commit()

## CADASTROS BASE

### CADUNIMEDIDA

In [ ]:
limpa_tabela(("cadunimedida",))

insert = cur_dest.prep("insert into cadunimedida(sigla,descricao) values(?,?)")

cur_orig.execute("select distinct substring(sigla,0,4) sigla, descricao from cadunidademedida where trim(sigla) <> ''")

for row in cur_orig:
    cur_dest.execute(insert, (row[0], row[1]))
commit()

### GRUPO E SUBGRUPO

In [ ]:
limpa_tabela(("cadsubgr", "cadgrupo"))

cur_orig.execute("""
select
	distinct
	to_char(grupo::integer, 'fm000') grupo,
	subgrupo,
	a.descricao
from
	cgasubgrupos a
union all
select distinct
	to_char(grupo::integer, 'fm000') grupo,
	concat(substring(mercadoria,1,1), substring(subgrupo,2,2)) subgrupo,
	b.descricao
	--, a.mercadoria
from
	cgc_vw_mercadorias a
left join cgasubgrupos b using (grupo, subgrupo)
where a.mercadoria not like '0%'                  
union all
	select
	distinct
	to_char(grupo::integer, 'fm000') grupo,
	'000',
	a.descricao
from
	cgagrupos a
order by
	2,
	1
""")

for row in cur_orig.fetchall():
	try:
		if row['subgrupo'] == '000':
			cur_dest.execute("insert into cadgrupo(grupo, nome) values(?, ?)", (row['grupo'], row['descricao'][:45]))
		else:
			cur_dest.execute("insert into cadsubgr(grupo, subgrupo, nome) values(?, ?, ?)", (row['grupo'], row['subgrupo'], row['descricao'][:45]))
		commit()
	except Exception as e:
		print(f"Erro ao inserir grupo/subgrupo {row['grupo']}/{row['subgrupo']}: {e} - {row.items()}")

cur_dest.execute("insert into cadgrupo(grupo, nome) values ('700', 'CONSUMO')")
cur_dest.execute("insert into cadgrupo(grupo, nome) values ('800', 'BEM')")
cur_dest.execute("insert into cadgrupo(grupo, nome) values ('900', 'SERVIÇOS')")
cur_dest.execute("insert into cadgrupo(grupo, nome) values ('000', 'SEM IDENTIFICAÇÃO')")
commit()

cur_dest.execute("insert into cadsubgr(grupo, subgrupo, nome) values ('700', '000', 'CONSUMO')")
cur_dest.execute("insert into cadsubgr(grupo, subgrupo, nome) values ('800', '000', 'BEM')")
cur_dest.execute("insert into cadsubgr(grupo, subgrupo, nome) values ('900', '000', 'SERVIÇOS')")
cur_dest.execute("insert into cadsubgr(grupo, subgrupo, nome) values ('000', '000', 'SEM IDENTIFICAÇÃO')")
commit()

### CADEST

In [ ]:
limpa_tabela(("cadest",))
cria_coluna("cadest", "key")

insert = cur_dest.prep("insert into cadest(grupo, subgrupo, codigo, cadpro, codreduz, disc1, ocultar, unid1, tipopro, usopro, key) values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)")

cur_orig.execute("""
with cte as (
select
	case 
		when tipo = 1 and nullif(trim(grupo), '') is null then '700'
		when tipo = 2 and nullif(trim(grupo), '') is null then '800'
		when tipo = 3 and nullif(trim(grupo), '') is null then '900'
		else to_char(nullif(trim(grupo), '')::int, 'fm000')
	end grupo,
	format(coalesce(nullif(trim(subgrupo), ''), '000'), 'fm000') subgrupo,
	trim(descricao) disc1,
	abs(hashtext(concat(trim(descricao),unidemb))) codreduz,
	cvi.unidemb unid1,
	tipo
from
	cgc_vw_itensprocesso cvi)
select distinct
	grupo,
	subgrupo,
	codreduz::text,
	disc1,
	unid1,
	tipo
from
	cte
union all
select
	'000',
	'000',
	b.codigocompleto,
	coalesce(a.nome, b.nome),
	c.sigla,
	1
from
	gapmaterialconsumo a
left join gcpespecificacao b on
b.id = a.especificacao 
left join cadunidademedida c on
a.unidadebasica = c.id 
where
	exists (
	select
		1
	from
		gap_vw_inventario x
	where
		x.materialconsumo = a.id)
union all 
select
	'000',
	'000',
	b.codigocompleto,
	coalesce(a.nome, b.nome),
	c.sigla,
	1
from
	gapmaterialconsumo a
left join gcpespecificacao b on
b.id = a.especificacao 
left join cadunidademedida c on
a.unidadebasica = c.id 
where
	not exists (
	select
		1
	from
		gap_vw_inventario x
	where
		x.materialconsumo = a.id)
""")

subgrupos_desdobrados = {k:[v, 0] for k, v in cur_dest.execute("select grupo||'.'||subgrupo, cast(subgrupo as integer) subgrupo_int from cadsubgr")}
max_subgrupo_grupo = {k: v for k, v in cur_dest.execute("select grupo, max(cast(subgrupo as integer)) from cadsubgr group by 1")}

tipo_uso_produto = {
	1: ['P', 'C'],
	2: ['P', 'P'],
	3: ['S', '']
}

for row in cur_orig.fetchall():
	key = f"{row['grupo']}.{row['subgrupo']}"
	subgrupo_int, codigo = subgrupos_desdobrados[key]
	codigo += 1

	if codigo > 999:
		subgrupo_ant = max_subgrupo_grupo[row['grupo']]
		subgrupo_int = max_subgrupo_grupo[row['grupo']] + 1
		codigo = 1
		subgrupos_desdobrados[key] = [subgrupo_int, codigo]
		max_subgrupo_grupo[row['grupo']] = subgrupo_int
	else:
		subgrupos_desdobrados[key][1] = codigo

	tipopro, usopro = tipo_uso_produto.get(row['tipo'], ['P', 'C'])

	cadpro = f'{row['grupo']}.{subgrupo_int:03}.{codigo:03}'

	cur_dest.execute(insert, (row['grupo'], f'{subgrupo_int:03}', f'{codigo:03}', cadpro, row['codreduz'], row['disc1'][:1024], 'N', row['unid1'], tipopro, usopro, key))
commit()

cur_dest.execute("""
INSERT INTO CADSUBGR (grupo, subgrupo, nome)
WITH CTE AS (
SELECT DISTINCT a.grupo, a.subgrupo, KEY FROM cadest a
WHERE NOT EXISTS (SELECT 1 FROM cadsubgr x WHERE a.grupo = x.grupo AND a.SUBGRUPO = x.SUBGRUPO))
SELECT a.GRUPO, a.SUBGRUPO, b.NOME FROM cte a
JOIN CADSUBGR b ON b.grupo||'.'||b.SUBGRUPO = a."KEY" 
""")
commit()

cadest = {k: v for k,v in cur_dest.execute("select codreduz, max(cadpro) from cadest group by codreduz").fetchall()}

### ALMOXARIFADOS

In [ ]:
limpa_tabela(("destino",))
cria_coluna("destino", "almox_pai")

insert = cur_dest.prep("""insert into destino(cod,desti,empresa,almox_pai) values(?,?,?,?)""")

cur_orig.execute("""
select
	to_char(id, 'fm000000000') cod,
	nome desti,
	g.unidadeprincipal pai,
    localfisico codccusto
from
	gapunidadealmox g  
""")

for row in cur_orig.fetchall():
    cur_dest.execute(insert, (row['cod'], row['desti'], empresa, row['pai']))
commit()

### CENTRO DE CUSTO

In [ ]:
limpa_tabela(("centrocusto",))
cria_coluna("centrocusto", "pai")

try:
    cur_dest.execute(f"insert into destino(cod,desti,empresa) values('000000000','CONVERSAO',{empresa})")
except fdb.DatabaseError as e:
    print(f"Erro ao inserir destino: {e}")
else:
    commit()

insert = cur_dest.prep("""
    insert into centrocusto (codccusto, descr, ccusto, ocultar, empresa, poder, orgao, destino, pai) values (?, ?, ?, ?, ?, ?, ?, ?, ?)
""")

cur_orig.execute("""
with CTE as (
    select
        id as codccusto,
        nome as descr,
        pai,
        versao,
        row_number() over (partition by id order by versao desc) as rn
    from
        pinhalpm.cadlocalfisico
)
select
    codccusto,
    descr,
    pai
from
    CTE a
where
    rn = 1;
""")

for row in cur_orig:
    cur_dest.execute(insert, (row['codccusto'], row['descr'], '001', 'N', empresa, '02', '01', '000000000', row['pai']))
cur_dest.execute(insert, (0, 'CONVERSÃO', '001', 'N', empresa, '02', '01', '000000000', None))
commit()



## COTAÇÃO



In [ ]:
cur_dest.execute("""
execute block as
    begin
    DELETE FROM vcadorc;
    DELETE FROM fcadorc;
    DELETE FROM icadorc;
    DELETE FROM cadorc;
end;
""")
commit()

### CADORC

In [ ]:
limpa_tabela(("cadorc",))

insert = cur_dest.prep("""
insert into cadorc
	(id_cadorc,
	num,
	ano,
	numorc,
	dtorc,
	descr,
	prioridade,
	obs,
	status,
	liberado,
	codccusto,
	liberado_tela,
	empresa) values
	(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
""")

cur_orig.execute("""
select distinct on (id_cadorc)
    id_cadorc,
    num,
    ano,
    numorc,
    dtorc,
    obs,
    descr,
    status,
    liberado,
    codccusto,
    liberado_tela
from (
    select
        concat(c.processo, EXTRACT(year from c.ANO)::INT%2000) id_cadorc,
        to_char(c.processo, 'fm00000') num,
        EXTRACT(year from c.ANO)::INT ano,
        concat(to_char(c.processo, 'fm00000/'),EXTRACT(year from c.ANO)::INT%2000) numorc,
        c.data dtorc,
        CONCAT(
            'SOLICITACOES: ',
            string_agg(
                distinct concat(to_char(a.solicitacao, 'fm00000/'),
                                EXTRACT(year from a.ANO)::INT%2000),
                ', ' order by concat(to_char(a.solicitacao, 'fm00000/'),
                                     EXTRACT(year from a.ANO)::INT%2000)
            )
        ) as obs,
        (array_agg(a.destinacao order by a.solicitacao))[1] as descr,
        case
            when c.status = 'A' then 'CA'
            when c.status = 'I' then 'AB'	
            else 'EC'
        end status,
        case when c.status = 'C' then 'S' else 'N' end liberado,
        coalesce(nullif(c.localentrega,-1),0) codccusto,
        case when c.status = 'C' then 'L' end liberado_tela
    from pinhalpm.cgcsolicitacoes a
    join (
        select distinct processo, ano, anosolicitacao, solicitacao
        from pinhalpm.cgc_vw_itenssolxprcp
    ) b 
        on a.ano = b.anosolicitacao
        and a.solicitacao = b.solicitacao
    join pinhalpm.cgcprocessocompra c 
        on b.ano = c.ano
        and b.processo = c.processo
    group by c.processo, c.ano, c.data, c.status, c.localentrega
) t
order by id_cadorc, liberado desc; -- garante que 'S' vem antes
""")

for row in cur_orig:
	cur_dest.execute(insert, (
		row['id_cadorc'],
		row['num'],
		row['ano'],
		row['numorc'],
		row['dtorc'],
		row['descr'][:1024],
		'NORMAL',
		row['obs'],
		row['status'],
		row['liberado'],
		row['codccusto'],
		row['liberado_tela'],
		empresa
	))
commit()

### ICADORC

In [ ]:
limpa_tabela(("icadorc",))

insert = cur_dest.prep("""
insert into icadorc (
    numorc,
    item,
    cadpro,
    qtd, 
    valor, 
    itemorc, 
    codccusto, 
    itemorc_ag,
    id_cadorc)
values(?,?,?,?,?,?,?,?,?)
""")

cur_orig.execute("""with CTE as (
select
	concat(to_char(processo, 'fm00000/'),EXTRACT(year from a.ANO) :: INT%2000) numorc,
    row_number() over (partition by processo, a.ano order by a.tipo, processo, a.ano, a.item) item,
    abs(hashtext(concat(trim(descricao),unidemb))) codreduz,
    quantidade,
    0 valorestimado,
    concat(a.processo, EXTRACT(year from a.ANO) :: INT%2000) id_cadorc
from
	cgc_vw_itensprocesso a)
select * from cte a""")

for row in cur_orig:
    try:
        key = str(row['codreduz'])
        cadpro = cadest[key]
    except Exception as e:
        print(f"Erro ao obter cadpro: {e}")
        break

    else:
        cur_dest.execute(insert, (
            row['numorc'],
            row['item'],
            cadpro,
            row['quantidade'],
            row['valorestimado'],
            row['item'],
            0,
            row['item'],
            row['id_cadorc']
        ))
commit()

cur_dest.execute("update icadorc a set a.codccusto = (select codccusto from cadorc b where a.id_cadorc = b.id_cadorc)")
commit()
    

### FCADORC

In [ ]:
limpa_tabela(("fcadorc",))

insert = cur_dest.prep("insert into fcadorc(numorc, codif, nome, valorc, id_cadorc) values (?,?,?,?,?)")

cur_orig.execute("""
select
	concat(to_char(a.processo, 'fm00000/'),EXTRACT(year from a.ANO) :: INT%2000) numorc,
	coalesce(fornecedor, abs(hashtext(concat(trim(fornaocadastrado))))) codif,
	coalesce(c.nome, fornaocadastrado) nome,
	coalesce(sum(valor*d.quantidade),0) valorc,
	concat(a.processo, EXTRACT(year from a.ANO) :: INT%2000) id_cadorc
from
	cgcprevia a
join cgcpreviaitens b using (processo, ano, previa)
left join cgcfornecedor c using (fornecedor)
left join cgc_vw_itensprocesso d using (tipo, processo, ano, item)
group by
	1,
	2,
	3,
	5
union all
select
	numorc,
	codif,
	substr(nome,1,70),
	valorc,
	id_cadorc
from (
select
	concat(to_char(a.processo, 'fm00000/'), extract(year from a.ANO) :: INT%2000) numorc,
	fornecedor codif,
	coalesce(c.nome, nome) nome,
	sum(valor) valorc,
	concat(a.processo, extract(year from a.ANO) :: INT%2000) id_cadorc
from
	cgc_vw_itensproposta a
left join cgcfornecedor c
		using (fornecedor)
where
	not exists (
	select
		1
	from
		cgcprevia b
	where
		a.processo = b.processo
		and a.ano = b.ano
		and a.fornecedor = b.fornecedor)
group by
	1,
	2,
	3,
	5)
""")

for row in cur_orig:
	cur_dest.execute(insert, (
		row['numorc'],
		row['codif'],
		row['nome'][:70],
		row['valorc'],
		row['id_cadorc']
	))
commit()

### VCADORC

In [ ]:
limpa_tabela(("vcadorc",))

insert = cur_dest.prep("""insert into vcadorc(numorc, item, codif, vlruni, vlrtot, ganhou, vlrganhou, classe, id_cadorc) values (?,?,?,?,?,?,?,?,?)""")

cur_orig.execute("""
select
    concat(to_char(a.processo, 'fm00000/'),EXTRACT(year from a.ANO) :: INT%2000) numorc,
    coalesce(fornecedor, abs(hashtext(concat(trim(fornaocadastrado))))) codif,
    dense_rank() over (
        partition by processo, a.ano
        order by a.tipo, a.item
    ) as item,
    valor vlruni,
    valor*quantidade vlrtot,
    'UN' classe,
    concat(a.processo, EXTRACT(year from a.ANO) :: INT%2000) id_cadorc
from
	cgcpreviaitens a
join cgcprevia b using (processo, ano, previa)
join (select processo, ano, tipo, item, quantidade from cgc_vw_itensprocesso) c using (processo, ano, tipo, item)
where valor is not null
union all
select
	concat(to_char(a.processo, 'fm00000/'),EXTRACT(year from a.ANO) :: INT%2000) numorc,
	fornecedor codif,
	dense_rank() over (
        partition by processo, a.ano
        order by a.tipoitem, a.item
    ) as item,
    coalesce(valor,0) vlruni,
    quantidade*coalesce(valor,0) vlrtot,
    'UN' classe,
    concat(a.processo, EXTRACT(year from a.ANO) :: INT%2000) id_cadorc
from
	cgc_vw_itensproposta a
join (select processo, ano, tipo tipoitem, item, quantidade from cgc_vw_itensprocesso) c using (processo, ano, tipoitem, item)
where
	not exists (
	select
		1
	from
		cgcprevia b
	where
		a.processo = b.processo
		and a.ano = b.ano
		and a.fornecedor = b.fornecedor) and not exists (select 1 from liclicitacao x where x.processocompra = a.processo and x.anoproc = a.ano)        
""")

for row in cur_orig:
    cur_dest.execute(insert, (
        row['numorc'],
        row['item'],
        row['codif'],
        row['vlruni'],
        row['vlrtot'],
        None,
        None,
        'UN',
        row['id_cadorc']
    ))
commit()

cur_dest.execute("""merge into vcadorc a
using (
    SELECT 
        id_cadorc, 
        item, 
        codif, 
        vlruni, 
        ROW_NUMBER() OVER (
            PARTITION BY id_cadorc, item 
            ORDER BY vlruni ASC, codif ASC
        ) AS rn
    FROM vcadorc
) b
on a.id_cadorc = b.id_cadorc
and a.item = b.item
and b.rn = 1
when matched then
    update set 
        a.vlrganhou = b.vlruni,
        a.ganhou = b.codif""")
commit()

## LICITAÇÕES

In [ ]:
#LIMPA LICITAÇÕES
cur_dest.execute("""
execute block as
    begin
    DELETE FROM regpreco;
    DELETE FROM regprecohis;
    DELETE FROM regprecodoc;
    DELETE FROM CADPROLIC_DETALHE_FIC;
    DELETE FROM CADPRO;
    DELETE FROM CADPRO_FINAL;
    DELETE FROM CADPRO_LANCE;
    DELETE FROM CADPRO_PROPOSTA;
    DELETE FROM PROLICS;
    DELETE FROM PROLIC;
    DELETE FROM CADPRO_STATUS;
    DELETE FROM CADLIC_SESSAO;
    DELETE FROM CADPROLIC_DETALHE;
    DELETE FROM CADPROLIC;
    DELETE FROM CADLIC;
    end;
""")
commit()

### CADLIC

In [ ]:
limpa_tabela(("cadlic",))

insert = cur_dest.prep("""
INSERT
		INTO
		cadlic(
		numlic,
		proclic,
		numero,
		ano,
		comp,
		licnova,
		liberacompra,
		discr,
		detalhe,
		registropreco,
		microempresa,
		numpro,
		discr7,
		datae,
		processo_data,
		dtadj,
		dthom,
		codtce,
		anomod,
		modlic,
		licit,
		codmod,
		dtpub,
		dtenc,
		valor,
		empresa,
		processo,
		processo_ano,
		dtreal,
        numorc,
        id_cadorc,
		tpapostilamento)
	VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?,
	?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?,
    ?, ?, 'I')
""")

cur_orig.execute("""
with cte as
(
select distinct
	concat(numero, extract(year from a.ano)::INT%2000) numlic,
	concat(to_char(numero, 'fm000000'), '/', extract(year from a.ano)::INT%2000) proclic,
	to_char(numero, 'fm000000') numero,
	extract(year from a.ano)::INT ano,
	case
		when percaprovado is not null then 3
		else 1
	end comp,
	1 licnova,
	'S' liberacompra,
	objeto discr,
	objetochar detalhe,
	case
		when modalidade in ('02','01','07','RP') or upper(objeto) ilike '%REGISTRO DE PREÇO%' then 'S'
		else 'N'
	end registropreco,
	'3' microempresa,
	nromodalidade numpro,
	'Menor Preco Unitario' discr7,
	digitacao datae,
	anoproc processo_data,
	datahomologacao dthom,
	b.codigoaudesp codtce,
	publicacao dtpub,
	valorestimado,
	processocompra,
	extract(year from ANOproc)::INT anoproc,
	coalesce(modalidade, 'DL') modlic,
	concat(to_char(processocompra, 'fm00000/'), extract(year from ANOproc)::INT%2000) numorc,
	concat(processocompra, extract(year from anoproc)::INT%2000) id_cadorc
from
	liclicitacao a
left join licaudesp b on
	a.numero = b.licitacao
	and a.ano = b.ano
	and b.tipoprocesso = 1) 
select
		*
from
	cte
""")

for row in cur_orig:
    info_modalidades = dict_modalidades.get(row['modlic'], {"Licit": "DISPENSA", "Modlic": "DI01", "Codmod": 1})

    licit = info_modalidades["Licit"]
    modlic = info_modalidades["Modlic"]
    codmod = info_modalidades["Codmod"]

    cur_dest.execute(insert, (
		row['numlic'], 
		row['proclic'],
		row['numero'],
		row['ano'],
		row['comp'],
		row['licnova'],
		row['liberacompra'],
		row['discr'],
		row['detalhe'],
		row['registropreco'],
		row['microempresa'],
		row['numpro'],
		row['discr7'],
		row['datae'],
		row['processo_data'],
		row['dthom'],
		row['dthom'],
		row['codtce'],
		row['ano'],
		modlic,
		licit,
		codmod,
		row['dtpub'],
		row['dthom'],
		row['valorestimado'],
		empresa,
		row['processocompra'],
		row['anoproc'],
		row['datae'],
		row['numorc'],
		row['id_cadorc']
	))
commit()

cur_dest.execute("""EXECUTE BLOCK AS
DECLARE VARIABLE DESCMOD VARCHAR(1024);
DECLARE VARIABLE CODMOD INTEGER;
BEGIN
    FOR
        SELECT CODMOD, DESCMOD FROM MODLIC INTO :CODMOD, :DESCMOD
    DO
    BEGIN
        UPDATE CADLIC SET LICIT = :DESCMOD where CODMOD = :CODMOD;
    END
END""")

cur_dest.execute("""UPDATE CADLIC SET FK_MODLICANO = (SELECT PK_MODLICANO FROM MODLICANO WHERE CODMOD = CADLIC.CODMOD AND ANOMOD = CADLIC.ANO AND CADLIC.EMPRESA = MODLICANO.EMPRESA) WHERE CODMOD IS NOT NULL""")
commit()

cur_dest.execute("UPDATE cadlicitacao SET nlicitacao = nlicitacao * 100000")
commit()

cur_dest.execute("UPDATE cadlicitacao SET nlicitacao = numlic")
commit()

cur_dest.execute("MERGE INTO cadlic a USING (select id_cadorc, numorc FROM cadorc ) b ON a.id_cadorc = b.id_cadorc WHEN MATCHED THEN UPDATE SET a.numorc = b.numorc")
commit()

cur_dest.execute("""MERGE
INTO
	cadorc a
		USING (
	SELECT
		id_cadorc,
		max(numlic) numlic,
		max(proclic) proclic
	FROM
		cadlic
	GROUP BY
		1) b
ON
	a.id_cadorc = b.id_cadorc
	WHEN MATCHED THEN
UPDATE
SET
	a.liberado = 'S',
	a.liberado_tela = 'L',
	a.status = 'LC',
	a.numlic = b.numlic,
	a.proclic = b.proclic""")
commit()

### CADLIC SESSAO

In [ ]:
cur_dest.execute("""INSERT INTO CADLIC_SESSAO (NUMLIC, SESSAO, DTREAL, HORREAL, COMP, DTENC, HORENC, SESSAOPARA, MOTIVO) 
SELECT L.NUMLIC, CAST(1 AS INTEGER), L.DTREAL, L.HORREAL, L.COMP, L.DTENC, L.HORENC, CAST('T' AS VARCHAR(1)), CAST('O' AS VARCHAR(1)) FROM CADLIC L 
WHERE numlic not in (SELECT FIRST 1 S.NUMLIC FROM CADLIC_SESSAO S WHERE S.NUMLIC = L.NUMLIC)""")
commit()

### PARTICIPANTES

In [ ]:
limpa_tabela(("prolics","prolic"))

insert_prolic = cur_dest.prep("insert into prolic(numlic,codif,nome,status) values(?,?,?,?)")
insert_prolics = cur_dest.prep("insert into prolics(sessao,numlic,codif,habilitado,status,cpf,representante) values(?,?,?,?,?,?,?)")

cur_orig.execute("""
select DISTINCT
	concat(licitacao,EXTRACT(YEAR FROM ano)::INT%2000) numlic,
	fornecedor,
	substr(nome, 1, 40) nome,
    cgccpf
from
	liclicfornecedor a
left join cgcfornecedor b using (fornecedor)
""")

for row in cur_orig:
    cur_dest.execute(insert_prolic, (row['numlic'], row['fornecedor'], to_cp1252_safe(row['nome']), 'A'))
    cur_dest.execute(insert_prolics, (1, row['numlic'], row['fornecedor'], 'S', 'A', row['cgccpf'], to_cp1252_safe(row['nome'])))
commit()

### CADPROLIC

In [ ]:
limpa_tabela(("cadprolic_detalhe_fic", "cadprolic_detalhe", "cadprolic"))
cria_coluna("cadprolic", "codant")

insert_cadprolic = cur_dest.prep("""
INSERT INTO
		cadprolic(item,
		item_mask,
		numorc,
		itemorc,
		cadpro,
		quan1,
		vamed1,
		vatomed1,
		codccusto,
		reduz,
		numlic,
        microempresa,
	    item_lc147,
		codant)
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
""")

cur_orig.execute("""
with cte as (
select
	concat(a.licitacao, extract(year from a.ano)::INT%2000) numlic,
	concat(to_char(a.licitacao, 'fm000000'), '/', extract(year from a.ano)::INT%2000) proclic,
	concat(to_char(processocompra, 'fm00000/'),EXTRACT(year from ANOproc)::INT%2000) numorc,
	itemlic item,
	exclusivome,
	itemreferenciame item_lc,
    qtde quan1,
	0 vaun1,
    0 vatomed1,
	codreduz,
	dense_rank() over (
        partition by processocompra, anoproc
        order by tipoitem, itempr
    ) as itemorc,
    concat(tipoitem,'.',itempr) codant
from
	lic_vw_itensxprcp a
left join (
select
	concat(to_char(processo, 'fm00000/'),EXTRACT(year from a.ANO) :: INT%2000) numorc,
    row_number() over (partition by processo, a.ano order by a.tipo, processo, a.ano, a.item) item,
    abs(hashtext(concat(trim(descricao),unidemb))) codreduz,
    quantidade,
    0 valorestimado,
    concat(a.processo, EXTRACT(year from a.ANO) :: INT%2000) id_cadorc,
    tipo,
    item  item_ant
from
	cgc_vw_itensprocesso a) b on b.id_cadorc = concat(processocompra, extract(year from anoproc)::INT%2000) and a.tipoitem = b.tipo and a.itempr = b.item_ant
left join 	(select
	licitacao,
	ano,
	item,
	case
		when l.tipoitem = 'M' then 1
		when l.tipoitem = 'B' then 2
		else 3
	end tipo,
	case when l.exclusivome = 1 then 'S' else 'N' end exclusivome,
	l.itemreferenciame,
	qtde
from
	liclicitens l) c on a.licitacao = c.licitacao and a.ano = c.ano and a.itemlic = c.item and a.tipoitem = c.tipo)
select * from cte
join (
	select distinct
		concat(a.numero, extract(year from a.ano)::INT%2000) numlic,
		coalesce(nullif(b.localentrega,-1),0) codccusto
	from
		liclicitacao a
	join cgcprocessocompra b on a.processocompra = b.processo and a.anoproc = b.ano) a 
using (numlic)
""")

for row in cur_orig:
    try:
        cadpro = cadest[str(row['codreduz'])]
    except KeyError:
        continue

    cur_dest.execute(insert_cadprolic, (row['item'], row['item'], row['numorc'], row['itemorc'], cadpro, row['quan1'], row['vaun1'], row['vatomed1'], row['codccusto'], 'N', row['numlic'], row['exclusivome'], row['item_lc'], row['codant']))
commit()

cur_dest.execute("""
	MERGE INTO cadprolic a
USING (
    SELECT numorc, item, vlruni
    FROM vcadorc
    WHERE ganhou = codif
      AND vlruni IS NOT NULL
) b
ON a.numorc = b.numorc
AND a.itemorc = b.item
WHEN MATCHED THEN
    UPDATE SET a.vamed1 = b.vlruni;
""")
commit()

cur_dest.execute("""
	INSERT INTO CADPROLIC_DETALHE (NUMLIC,item,CADPRO,quan1,VAMED1,VATOMED1,marca,CODCCUSTO,ITEM_CADPROLIC, item_lc147)
	select numlic, item, cadpro, quan1, vamed1, vatomed1, marca, codccusto, item, item_lc147 from cadprolic b where
	not exists (select 1 from cadprolic_detalhe c where b.numlic = c.numlic and b.item = c.item);
""")
commit()

cur_dest.execute("""
	insert into 
	cadprolic_detalhe_fic (numlic, item, codigo, qtd, valor, qtdadt, valoradt, codccusto, qtdmed, valormed, tipo) 
	select numlic, item, item, quan1, vatomed1, quan1, vatomed1, codccusto, quan1, vatomed1, 'C' from cadprolic b where
	not exists (select 1 from cadprolic_detalhe_fic c where b.numlic = c.numlic and b.item = c.item);
""")
commit()

### PROPOSTA

In [ ]:
limpa_tabela(("cadpro", "cadpro_final", "cadpro_proposta"))

insert = cur_dest.prep("""
INSERT
    INTO
    cadpro_proposta(sessao,
    codif,
    item,
    itemp,
    quan1,
    vaun1,
    vato1,
    numlic,
    status,
    subem,
    marca,
    itemlance)
VALUES(?,?,?,?,?,?,?,?,?,?,?,?)""")

cur_orig.execute("""
select
	fornecedor codif,
	item,
	qtde quan1,
	valor vaun1,
	coalesce(valortotal, qtde*valor) vato1,
	concat(licitacao,EXTRACT(YEAR FROM ano)::INT%2000) numlic,
	'C' status,
	'S' itemlance
from
	liclicconitens
""")           

for row in cur_orig:
    cur_dest.execute(insert, (1, row['codif'], row['item'], row['item'], row['quan1'], row['vaun1'], row['vato1'], row['numlic'], row['status'], 1, None, row['itemlance']))
commit()

### CADPRO LANCE

In [ ]:
cur_dest.execute("""insert into cadpro_lance (sessao, rodada, codif, itemp, vaunl, vatol, status, subem, numlic)
	SELECT sessao, 1 rodada, CODIF, ITEMP, VAUN1, VATO1, 'F' status, SUBEM, numlic FROM CADPRO_PROPOSTA cp where subem = 1 and not exists
	(select 1 from cadpro_lance cl where cp.codif = cl.codif and cl.itemp = cp.itemp and cl.numlic = cp.numlic)""")
commit()

### CADPRO FINAL

In [ ]:
cur_dest.execute("""INSERT into cadpro_final (numlic, ult_sessao, codif, itemp, vaunf, vatof, STATUS, subem)
	SELECT numlic, sessao, codif, itemp, vaun1, vato1, CASE WHEN status = 'F' THEN 'C' ELSE status end, subem FROM cadpro_proposta
	WHERE NOT EXISTS (SELECT 1 FROM cadpro_final f WHERE f.numlic = cadpro_proposta.numlic AND f.itemp = cadpro_proposta.itemp AND f.codif = cadpro_proposta.codif)""")
commit()

cur_dest.execute("""update cadlic a SET comp = 3 WHERE comp = 1 AND EXISTS (SELECT 1 FROM cadpro_proposta x WHERE subem = 1 AND x.numlic = a.numlic) AND NOT EXISTS (SELECT 1 FROM cadpro x WHERE x.numlic = a.numlic)""")
commit()

### CADPRO_STATUS

In [ ]:
cur_dest.execute("""
    INSERT INTO cadpro_status (numlic, sessao, itemp, item, telafinal)
    SELECT b.NUMLIC, 1 AS sessao, a.item, a.item, 'I_ENCERRAMENTO'
    FROM CADPROLIC a
    JOIN cadlic b ON a.NUMLIC = b.NUMLIC
    WHERE NOT EXISTS (
        SELECT 1
        FROM cadpro_status c
        WHERE a.numlic = c.numlic
    )
    AND EXISTS (
        SELECT 1
        FROM cadlic d
        WHERE d.numlic = a.numlic AND d.comp = 3
    );
""")
commit()

### CADPRO

In [ ]:
cria_coluna("cadpro", "codant")

cur_dest.execute("""
INSERT INTO CADPRO (
  CODIF, CADPRO, QUAN1, VAUN1, VATO1, SUBEM, STATUS,
  ITEM, NUMORC, ITEMORCPED, CODCCUSTO, FICHA, ELEMENTO, DESDOBRO,
  NUMLIC, ULT_SESSAO, ITEMP, QTDADT, QTDPED, VAUNADT, VATOADT,
  PERC, QTDSOL, ID_CADORC, VATOPED, VATOSOL, TPCONTROLE_SALDO,
  QTDPED_FORNECEDOR_ANT, VATOPED_FORNECEDOR_ANT, MARCA
)
WITH dados AS (
  SELECT
    a.CODIF,
      c.CADPRO,
      c.ITEM,
      d.NUMORC,
      c.CODCCUSTO,
      c.FICHA,
      c.ELEMENTO,
      c.DESDOBRO,
      a.NUMLIC,
      b.ITEMP,
      d.ID_CADORC,
      a.marca,
      a.quan1  AS qtdunit,
      a.VAUN1 AS vaun,
      a.vato1 AS vatotal
  FROM CADPRO_PROPOSTA a
  JOIN CADPRO_STATUS b
    ON b.NUMLIC = a.NUMLIC
    AND b.ITEMP  = a.ITEMP
    AND b.SESSAO = a.SESSAO
  JOIN CADPROLIC_DETALHE c
    ON c.NUMLIC = a.NUMLIC
    AND b.ITEMP  = c.ITEM_CADPROLIC
  JOIN CADLIC d
    ON d.NUMLIC = a.NUMLIC
  WHERE a.SUBEM = 1
    AND a.STATUS = 'C'
)
SELECT
    CODIF,
    CADPRO,
    qtdunit,
    vaun,
    vatotal,
    1,
    'C',
    ITEM,
    NUMORC,
    ITEM,
    CODCCUSTO,
    FICHA,
    ELEMENTO,
    DESDOBRO,
    NUMLIC,
    1,
    ITEMP,
    qtdunit,
    0,
    vaun,
    vatotal,
    0,
    0,
    ID_CADORC,
    0,
    0,
    'Q',
    0,
    0,
    marca
FROM dados d
WHERE NOT EXISTS (
    SELECT 1
    FROM CADPRO cp
    WHERE cp.NUMLIC = d.NUMLIC
      AND cp.ITEM   = d.ITEM
      AND cp.CODIF  = d.CODIF
)""")
commit()

cur_dest.execute("""
MERGE INTO cadpro a USING (SELECT numlic, item, codant FROM cadprolic) b
ON a.numlic = b.numlic AND a.item = b.item 
WHEN MATCHED THEN UPDATE SET a.codant = b.codant
""")
commit()

### ADITIVOS

In [ ]:
itens_processo = {k:v for k,v in cur_dest.execute("select numlic||'.'||codif||'.'||codant, item from cadpro a JOIN cadest b USING (cadpro)").fetchall()}
itens_processo_sem_codif = {k:v for k,v in cur_dest.execute("select numlic||'.'||codant, item from cadpro a JOIN cadest b USING (cadpro)").fetchall()}

cur_dest.execute("UPDATE CADPRO SET QTDADT = QUAN1, VAUNADT = VAUN1, VATOADT = VATO1 WHERE QTDADT <> QUAN1 OR VATOADT <> VATO1")
commit()

cur_orig.execute("""
with ccp_vw_contr_adt_item as (
SELECT a.idcontrato,
    0 AS idaditamento,
    '1_CREDENCIAMENTO'::character varying(100) AS tiporegistro,
    a.ano,
    a.tipo,
    a.modelo,
    a.contrato,
    0 AS aditamento,
    0 AS apostilamento,
    NULL::character varying(30) AS tipoaditamento,
    NULL::character varying(30) AS detalheaditamento,
    NULL::character varying(30) AS tipo_alteracao,
    NULL::character varying(30) AS validacalculo,
    NULL::integer AS iteminserido,
    a.tipoitem,
    a.itemcontrato,
    0 AS itemaditamento,
    a.anoprocesso,
    a.processo,
    a.itemprocesso,
    a.descricao,
    lit.qtde AS quantidade,
    lit.valor,
    b.inicio AS data,
    b.contratado,
    b.inicio,
    b.fim,
    a.status,
    b.datapublicacao,
    NULL::numeric(15,7) AS valorpsolicitacao,
    a.anoprocessoaux,
    a.processoaux,
    a.itemprocessoaux
   FROM pinhalpm.ccpcontratoitens a
     JOIN pinhalpm.ccpcontratos b ON b.id = a.idcontrato
     JOIN pinhalpm.ccpcontratoadici c ON b.id = c.idcontrato AND c.saldoprocessoaux = 1
     JOIN pinhalpm.lic_vw_itensxprcp li ON li.anoproc = a.anoprocessoaux AND li.processocompra = a.processoaux AND li.tipoitem = a.tipoitem AND li.itempr = a.itemprocessoaux
     JOIN pinhalpm.liclicitens lit ON li.ano = lit.ano AND li.licitacao = lit.licitacao AND li.itemlic = lit.item
UNION ALL
 SELECT a.idcontrato,
    0 AS idaditamento,
    '1_CONTRATO'::character varying(100) AS tiporegistro,
    a.ano,
    a.tipo,
    a.modelo,
    a.contrato,
    0 AS aditamento,
    0 AS apostilamento,
    NULL::character varying(30) AS tipoaditamento,
    NULL::character varying(30) AS detalheaditamento,
    NULL::character varying(30) AS tipo_alteracao,
    NULL::character varying(30) AS validacalculo,
    NULL::integer AS iteminserido,
    a.tipoitem,
    a.itemcontrato,
    0 AS itemaditamento,
    a.anoprocesso,
    a.processo,
    a.itemprocesso,
    a.descricao,
    a.quantidade,
    a.valor,
    b.inicio AS data,
    b.contratado,
    b.inicio,
    b.fim,
    a.status,
    b.datapublicacao,
    NULL::numeric(15,7) AS valorpsolicitacao,
    a.anoprocessoaux,
    a.processoaux,
    a.itemprocessoaux
   FROM pinhalpm.ccpcontratoitens a
     JOIN pinhalpm.ccpcontratos b ON b.id = a.idcontrato
  WHERE NOT (EXISTS ( SELECT 1
           FROM pinhalpm.ccpcontratoadici ad
          WHERE a.idcontrato = ad.idcontrato AND ad.saldoprocessoaux = 1))
UNION ALL
 SELECT a.idcontrato,
    b.idaditamento,
        CASE b.apostilamento
            WHEN 1 THEN '2_APOSTILAMENTO'::text
            ELSE '2_ADITAMENTO'::text
        END::character varying(100) AS tiporegistro,
    a.ano,
    a.tipo,
    a.modelo,
    a.contrato,
    b.aditamento,
    b.apostilamento,
        CASE c.tipoaditamento
            WHEN 1 THEN 'PRAZO'::text
            WHEN 2 THEN 'QTDE_VLR'::text
            WHEN 3 THEN 'INCORPORACAO'::text
            WHEN 4 THEN 'ALT_FORNECEDOR'::text
            WHEN 5 THEN 'CISAO'::text
            WHEN 6 THEN 'PRAZO_EXECUCAO'::text
            WHEN 99 THEN 'OUTROS'::text
            ELSE 'ERRO'::text
        END::character varying(30) AS tipoaditamento,
    COALESCE(b.operacao,
        CASE c.detalhetpadt
            WHEN 1 THEN 'ACRESCIMO'::text
            WHEN 2 THEN 'DECRESCIMO'::text
            WHEN 3 THEN 'REAJUSTE'::text
            WHEN 4 THEN 'RECOMPOSICAO'::text
            WHEN 5 THEN 'ALT_ESPECPROJETO'::text
            ELSE 'ACRESCIMO'::text
        END::character varying)::character varying(30) AS detalheaditamento,
        CASE c.qtdevlr
            WHEN 'A'::text THEN 'QTDE_VALOR'::text
            WHEN 'V'::text THEN 'VALOR'::text
            WHEN 'Q'::text THEN 'QTDE'::text
            ELSE NULL::text
        END::character varying(30) AS tipo_alteracao,
        CASE c.vlrprazocalcsaldo
            WHEN 1 THEN 'CALCULA'::text
            ELSE 'NAO_CALCULA'::text
        END::character varying(30) AS validacalculo,
    b.iteminserido,
    a.tipoitem,
    a.itemcontrato,
    b.item AS itemaditamento,
    a.anoprocesso,
    a.processo,
    a.itemprocesso,
    a.descricao,
    b.quantidade,
    b.valor,
    c.data,
    c.contratado,
    c.inicio,
    c.fim,
    c.status,
    c.datapublicacao,
    b.valorpsolicitacao,
    a.anoprocessoaux,
    a.processoaux,
    a.itemprocessoaux
   FROM pinhalpm.ccpcontratoitens a
     JOIN pinhalpm.ccpcontratosadtit b ON a.idcontrato = b.idcontrato AND a.tipoitem = b.tipoitem AND a.itemcontrato = b.itemcontrato
     JOIN pinhalpm.ccpcontratosadt c ON a.idcontrato = c.idcontrato AND c.id = b.idaditamento
  WHERE NOT (EXISTS ( SELECT 1
           FROM pinhalpm.ccpcontratoadici ad
          WHERE a.idcontrato = ad.idcontrato AND ad.saldoprocessoaux = 1)))
select
	concat(b.numero, extract(year from b.ano)::INT%2000) numlic,
	tipoitem||'.'||itemprocesso codant,
	quantidade,
	valor,
	contratado codif,
	contrato
from
	pinhalpm.ccp_vw_contr_adt_item a
join liclicitacao b on a.processo = b.processocompra and a.anoprocesso = b.anoproc
where tiporegistro = '2_ADITAMENTO'
""")

update = cur_dest.prep("update cadpro set QTDADT = QTDADT+?, VAUNADT = ?, VATOADT = VATOADT+(?*?) where numlic = ? and item = ?")

for row in cur_orig:
	try:
		key = f'{row["numlic"]}.{row["codif"]}.{row["codant"]}'
		item = itens_processo[key]
	except:
		try:
			key = f'{row["numlic"]}.{row["codant"]}'
			item = itens_processo_sem_codif[key]
		except:
			print(f'Item nao encontrado para aditamento: {row["numlic"]} - {row["codif"]} - {row["codant"]}')
			continue
	
	cur_dest.execute(update, (row['quantidade'], row['valor'], row['quantidade'], row['valor'], row['numlic'], item))
	commit()

### REGPRECO

In [ ]:
limpa_tabela(("regprecodoc", "regpreco", "regprecohis"))

cur_dest.execute("""
EXECUTE BLOCK AS  
    BEGIN  
    INSERT INTO REGPRECODOC (NUMLIC, CODATUALIZACAO, DTPRAZO, ULTIMA)  
    SELECT DISTINCT A.NUMLIC, 0, DATEADD(1 YEAR TO A.DTHOM), 'S'  
    FROM CADLIC A WHERE A.REGISTROPRECO = 'S' AND A.DTHOM IS NOT NULL  
    AND NOT EXISTS(SELECT 1 FROM REGPRECODOC X  
    WHERE X.NUMLIC = A.NUMLIC);  

    INSERT INTO REGPRECO (COD, DTPRAZO, NUMLIC, CODIF, CADPRO, CODCCUSTO, ITEM, CODATUALIZACAO, QUAN1, VAUN1, VATO1, QTDENT, SUBEM, STATUS, ULTIMA)  
    SELECT B.ITEM, DATEADD(1 YEAR TO A.DTHOM), B.NUMLIC, B.CODIF, B.CADPRO, B.CODCCUSTO, B.ITEM, 0, B.QUAN1, B.VAUN1, B.VATO1, 0, B.SUBEM, B.STATUS, 'S'  
    FROM CADLIC A INNER JOIN CADPRO B ON (A.NUMLIC = B.NUMLIC) WHERE A.REGISTROPRECO = 'S' AND A.DTHOM IS NOT NULL  
    AND NOT EXISTS(SELECT 1 FROM REGPRECO X  
    WHERE X.NUMLIC = B.NUMLIC AND X.CODIF = B.CODIF AND X.CADPRO = B.CADPRO AND X.CODCCUSTO = B.CODCCUSTO AND X.ITEM = B.ITEM);  

    INSERT INTO REGPRECOHIS (NUMLIC, CODIF, CADPRO, CODCCUSTO, ITEM, CODATUALIZACAO, QUAN1, VAUN1, VATO1, SUBEM, STATUS, MOTIVO, MARCA, NUMORC, ULTIMA)  
    SELECT B.NUMLIC, B.CODIF, B.CADPRO, B.CODCCUSTO, B.ITEM, 0, B.QUAN1, B.VAUN1, B.VATO1, B.SUBEM, B.STATUS, B.MOTIVO, B.MARCA, B.NUMORC, 'S'  
    FROM CADLIC A INNER JOIN CADPRO B ON (A.NUMLIC = B.NUMLIC) WHERE A.REGISTROPRECO = 'S' AND A.DTHOM IS NOT NULL  
    AND NOT EXISTS(SELECT 1 FROM REGPRECOHIS X  
    WHERE X.NUMLIC = B.NUMLIC AND X.CODIF = B.CODIF AND X.CADPRO = B.CADPRO AND X.CODCCUSTO = B.CODCCUSTO AND X.ITEM = B.ITEM);  
END;""")
commit()

## PEDIDOS

### ANTERIORES AO EXERCÍCIO

In [ ]:
limpa_tabela(("cadpro_saldo_ant", "regpreco_saldo_ant"))

itens_processo = {k:[v1,v2] for k,v1,v2 in cur_dest.execute("select numlic||'.'||codif||'.'||codant, cadpro, item from cadpro a JOIN cadest b USING (cadpro)").fetchall()}
itens_processo_sem_codif = {k:[v1,v2] for k,v1,v2 in cur_dest.execute("select numlic||'.'||codant, cadpro, item from cadpro a JOIN cadest b USING (cadpro)").fetchall()}

insert = cur_dest.prep("""
insert into cadpro_saldo_ant (
    ano,
    numlic,
    item,
    cadpro,
    qtdped,
    vatoped)
values (?,?,?,?,?,?)
""")

cur_orig.execute("""
with cte as
(select
	to_char(pedido, 'fm00000') num,
	extract(year from a.ano)::int%2000 ano,
	data,
	fornecedor codif,
	id_cadped,
	concat(c.numero, extract(year from c.ano)::INT%2000) numlic,
	destinacao obs,
	codlocal codccusto,
	item codant,
	sum(quantidade) qtd,
	avg(valor) prcunt,
	sum(vlrtotal) prctot
from
	cgcpedidos a
join (
select
	concat(pedido,extract(year from ano)::int%2000) id_cadped,
	concat(tipo,'.',itemprocesso) item,
	abs(hashtext(concat(trim(descricao),unidemb))) codreduz,
	quantidade,
	valor,
	vlrtotal,
	pedido,
	ano
from
	cgc_vw_itenspedbase order by item) b using (pedido, ano)
join liclicitacao c on c.processocompra = a.processo and a.anoprocesso = c.anoproc
where extract(year from data) < '2025'
group by 1,2,3,4,5,6,7,8,9
order by 1, codant)
select numlic, codif, codant, sum(qtd) qtdped, sum(prctot) vatoped from cte group by 1,2,3
""")

for row in cur_orig:
    try:
        key = f'{row["numlic"]}.{row["codif"]}.{row["codant"]}'
        item = itens_processo[key]
    except:
        try:
            key = f'{row["numlic"]}.{row["codant"]}'
            item = itens_processo_sem_codif[key]
        except:
            continue
        
    try:
        cur_dest.execute(insert, (
			exercicio,
			row['numlic'],
			item[1],
			item[0],
			row['qtdped'],
			row['vatoped']
		))
        commit()
    except fdb.DatabaseError as e:
        if e.args[1] == -803: 
            cur_dest.execute("update cadpro_saldo_ant set qtdped = qtdped + ?, vatoped = vatoped + ? where numlic = ? and item = ?", (row['qtdped'], row['vatoped'], row['numlic'], item[1])) 
            commit()
        else:
            raise

cur_dest.execute("""
insert into regpreco_saldo_ant (ano, item, cadpro, qtdent, numlic, vatoent) select ano, item, cadpro, qtdped, numlic, vatoped from cadpro_saldo_ant 
where exists (select 1 from cadlic where cadpro_saldo_ant.numlic = cadlic.numlic and cadlic.registropreco = 'S')
""")
commit()

### DO EXERCÍCIO

In [ ]:
limpa_tabela(("icadped", "cadped"))

itens_processo = {k:[v1,v2] for k,v1,v2 in cur_dest.execute("select numlic||'.'||codif||'.'||codant, cadpro, item from cadpro a JOIN cadest b USING (cadpro)").fetchall()}
itens_processo_sem_codif = {k:[v1,v2] for k,v1,v2 in cur_dest.execute("select numlic||'.'||codant, cadpro, item from cadpro a JOIN cadest b USING (cadpro)").fetchall()}

itens_cotacao = {k:[v1,v2] for k,v1,v2 in cur_dest.execute("select id_cadorc||'.'||codif||'.'||codant, cadpro, item from cadpro a JOIN cadest b USING (cadpro)").fetchall()}

inser_cadped = cur_dest.prep("insert into cadped (numped, num, ano, datped, codif, id_cadped, empresa, numlic, obs, codccusto, id_cadorc) values (?,?,?,?,?,?,?,?,?,?,?)")
insert_icadped = cur_dest.prep("insert into icadped (numped, item, cadpro, qtd, prcunt, prctot, codccusto, id_cadped) values (?,?,?,?,?,?,?,?)")

cur_orig.execute("""
select
	to_char(pedido, 'fm00000') num,
	extract(year from a.ano)::int%2000 ano,
	data,
	fornecedor codif,
	id_cadped,
	concat(c.numero, extract(year from c.ano)::INT%2000) numlic,
	destinacao obs,
	codlocal codccusto,
	item codant,
	sum(quantidade) qtd,
	avg(valor) prcunt,
	sum(vlrtotal) prctot,
    null id_cadorc
from
	cgcpedidos a
join (
select
	concat(pedido,extract(year from ano)::int%2000) id_cadped,
	concat(tipo,'.',itemprocesso) item,
	abs(hashtext(concat(trim(descricao),unidemb))) codreduz,
	quantidade,
	valor,
	vlrtotal,
	pedido,
	ano
from
	cgc_vw_itenspedbase order by item) b using (pedido, ano)
join liclicitacao c on c.processocompra = a.processo and a.anoprocesso = c.anoproc
where extract(year from data) ='2025'
group by 1,2,3,4,5,6,7,8,9
union all
select
	to_char(pedido, 'fm00000') num,
	extract(year from a.ano)::int%2000 ano,
	data,
	fornecedor codif,
	id_cadped,
	null,
	destinacao obs,
	codlocal codccusto,
	item_seq::text codant,
	sum(b.quantidade) qtd,
	avg(b.valor) prcunt,
	sum(b.vlrtotal) prctot,
	concat(a.processo, EXTRACT(year from a.ANOprocesso)::INT%2000) id_cadorc
from
	cgcpedidos a
join (
select
	concat(pedido,extract(year from ano)::int%2000) id_cadped,
	tipo,
	item,
	abs(hashtext(concat(trim(descricao),unidemb))) codreduz,
	quantidade,
	valor,
	vlrtotal,
	pedido,
	ano
from
	cgc_vw_itenspedbase order by item) b using (pedido, ano)
join (select processo, ano, tipo, item, dense_rank() over ( partition by processo, a.ano order by a.tipo, a.item ) as item_seq, quantidade from cgc_vw_itensprocesso a) c 
on c.processo = a.processo 
and c.ano = a.anoprocesso
and b.item = c.item
and b.tipo = c.tipo
where extract(year from data) ='2025'
and not exists (select 1 from liclicitacao x where x.anoproc = a.anoprocesso and a.processo = x.processocompra)
group by 1,2,3,4,5,6,7,8,9,13
order by 1, codant
""")

id_cadped = 0

for row in cur_orig:
    numped = f'{row["num"]}/{row["ano"]}'
    if row['id_cadped'] != id_cadped:
        id_cadped = row['id_cadped']
        cur_dest.execute(inser_cadped, (
            numped,
            row['num'],
            f'20{row["ano"]}',
            row['data'],
            row['codif'],
            id_cadped,
            empresa,
            row['numlic'],
            row['obs'],
            row['codccusto'],
            row['id_cadorc']
        ))
        commit()
    
    if not row['id_cadorc']:
        try:
            key = f'{row["numlic"]}.{row["codif"]}.{row["codant"]}'
            item = itens_processo[key]
        except:
            print(f'Chave não encontrada: {key} - pedido: {row["num"]}')
            try:
                key = f'{row["numlic"]}.{row["codant"]}'
                item = itens_processo_sem_codif[key]
            except:
                print(f'Chave não encontrada sem codif: {key} - pedido: {row["num"]}')
                continue
    else:
        key = f'{row["id_cadorc"]}.{row["codif"]}.{row["codant"]}'
        item = itens_cotacao[key]

    try:
        cur_dest.execute(insert_icadped, (
            numped,
            item[1],
            item[0],
            row['qtd'],
            row['prcunt'],
            row['prctot'],
            row['codccusto'],
            id_cadped
        ))
    except:
        print(f'Erro ao inserir pedido {row['num']}')
        continue
commit()

cur_dest.execute("""
UPDATE cadped a SET a.codatualizacao_rp = (SELECT max(codatualizacao) 
FROM regprecodoc x WHERE x.numlic = a.numlic)
""")

# 📦 ALMOXARIFADO
Extração, tratamento e carregamentos dos dados referentes ao módulo de almoxarifado

### SALDO INICIAL

In [ ]:
limpa_tabela(("icadreq where id_requi = 0", "requi where id_requi = 0"))
cur_dest.execute("alter trigger TAU_ESTOQUE active")
commit()

cur_dest.execute(f"""
INSERT
    INTO
    requi (empresa,
    id_requi,
    requi,
    num,
    ano,
    destino,
    codccusto,
    datae,
    dtlan,
    dtpag,
    entr,
    said,
    comp,
    codif,
    entr_said)
VALUES (
    {empresa},
    0,
    '000000/{exercicio[-2:]}',
    '000000',
    {exercicio},
    '000000000',
    0,
    '31.12.{int(exercicio)-1}',
    '31.12.{int(exercicio)-1}',
    null,
    'S',
    'N',
    'P',
    0,
    null)
""")

insert_icadreq = cur_dest.prep("""
    insert into icadreq (id_requi, requi, codccusto, empresa, item, quan1, quan2, vaun1, vaun2, vato1, vato2, cadpro, destino) values (?,?,?,?,?,?,?,?,?,?,?,?,?)
""")

cur_orig.execute("""
select
	to_char(gvi.almoxarifado, 'fm000000000') destino,
	localfisico codccusto,
	gvi.codigocompleto,
	gvi.quantidadeinicial quan1,
	valorinicial/case when quantidadeinicial = 0 then 1 else quantidadeinicial end vaun1,
	gvi.valorinicial vato1,
    row_number() over (order by gvi.almoxarifado, codigocompleto) item
from
	gap_vw_inventario gvi
join gapunidadealmox gua on gvi.almoxarifado = gua.id 
where
    gvi.dataexclusao is null
    --and gvi.quantidadeinicial <> 0
    and fim >= '2025-01-01'
    and fim < '2025-02-01'
order by 1,3
""")

for row in cur_orig:
    try:
        cadpro = cadest[str(row['codigocompleto'])]
    except:
        print(f'Erro ao obter cadpro para o item {row["codigocompleto"]}. Verifique se o item está cadastrado.')
        break

    cur_dest.execute(insert_icadreq, (0, f'000000/{exercicio[-2:]}', row['codccusto'], empresa, row['item'], row['quan1'], 0, row['vaun1'], 0, row['vato1'], 0, cadpro, row['destino']))
commit()

### REQUISIÇÕES

In [ ]:
limpa_tabela(("icadreq where id_requi <> 0", "requi where id_requi <> 0"))

insert_requi = cur_dest.prep("""
INSERT
    INTO
    requi (empresa,
    id_requi,
    requi,
    num,
    ano,
    destino,
    codccusto,
    datae,
    dtlan,
    dtpag,
    entr,
    said,
    comp,
    codif,
    entr_said,
    docum)
VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
""")

insert_icadreq = cur_dest.prep("""
    insert into icadreq (id_requi, requi, codccusto, empresa, item, quan1, quan2, vaun1, vaun2, vato1, vato2, cadpro, destino) values (?,?,?,?,?,?,?,?,?,?,?,?,?)
""")

cur_orig.execute(f"""
select
	a.id id_requi,
	concat(to_char(a.id, 'fm000000/'),extract(year from datahora)::int%2000) requi,
	to_char(a.id, 'fm000000') num,
	extract(year from datahora) ano,
	b.cod destino,
	localfisico codccusto,
	dataregistro::date data,
	'E' tipo,
	fornecedor codif,
	codigo doc,
	row_number() over (partition by a.id order by codigocompleto) item,
    codigocompleto,
	abs(a.quantidade) qtd,
	abs(a.valorunitario) vaun,
	abs(a.total_item) vato
from
	gap_vw_movimentacao a
left join (select a.id, to_char(almoxarifado, 'fm000000000') cod, x.localfisico from gapestoque a left join (select id, localfisico from pinhalpm.gapunidadealmox) x on x.id = a.almoxarifado) b on b.id = a.estoque
left JOIN pinhalpm.gcpentradaitem f ON a.entradamatitem = f.id
     LEFT JOIN pinhalpm.gcpdocentradaitem g ON f.docentradaitem = g.id
     LEFT JOIN pinhalpm.gcpdocentrada h ON g.documentoentrada = h.id
where
	dtype = 'ENTRADA'
	and extract(year from dataregistro) = {exercicio}
union all
select
	a.id id_requi,
	concat(to_char(a.id, 'fm000000/'),extract(year from datahora)::int%2000) requi,
	to_char(a.id, 'fm000000') num,
	extract(year from datahora) ano,
	b.cod destino,
	localfisico codccusto,
	dataregistro::date data,
	'S' tipo,
	null codif,
	null,
	row_number() over (partition by a.id order by codigocompleto) item,
    codigocompleto,
	abs(quantidade) qtd,
	abs(valorunitario) vaun,
	abs(total_item) vato
from
	gap_vw_movimentacao a
left join (select a.id, to_char(almoxarifado, 'fm000000000') cod, x.localfisico from gapestoque a left join (select id, localfisico from pinhalpm.gapunidadealmox) x on x.id = a.almoxarifado) b on b.id = a.estoque
where
	dtype = 'SAIDA'
	and extract(year from dataregistro) = {exercicio}
""")

requi_ant = 0
for row in cur_orig:
    if row['tipo'] == 'E':
        quan1 = row['qtd']
        vaun1 = row['vaun']
        vato1 = row['vato']
        
        quan2 = 0
        vaun2 = 0
        vato2 = 0

        entr = 'S'
        said = 'N'
        entr_said = 'N'
    else:
        quan1 = 0
        vaun1 = 0
        vato1 = 0

        quan2 = row['qtd']
        vaun2 = row['vaun']
        vato2 = row['vato']

        entr = 'N'
        said = 'S'
        entr_said = 'N'

    if row['id_requi'] != requi_ant:
        cur_dest.execute(insert_requi, (
            empresa,
            row['id_requi'],
            row['requi'],
            row['num'],
            row['ano'],
            row['destino'],
            row['codccusto'],
            None,
            row['data'],
            None,
            entr,
            said,
            'P',
            row['codif'],
            entr_said,
            row['doc']
        ))
        commit()
        requi_ant = row['id_requi']

    try:
        cadpro = cadest[str(row['codigocompleto'])]
    except:
        print(f'Erro ao obter cadpro para o item {row["codigocompleto"]}. Verifique se o item está cadastrado.')
        break

    cur_dest.execute(insert_icadreq, (
        row['id_requi'],
        row['requi'],
        row['codccusto'],
        empresa,
        row['item'],
        quan1,
        quan2,
        vaun1,
        vaun2,
        vato1,
        vato2,
        cadpro,
        row['destino']
    ))
    commit()

# cur_dest.execute("""
# execute block as
# declare variable produto varchar(11);
# begin
# for
# select cadpro from icadreq
# where requi = '000000/25'
# and quan1 = 0
# and quan2 = 0
# and vato1 = 0
# and vato2 = 0
# into :produto
 
# do
 
# Begin
# delete from estoque_destino where cadpro = :produto;
# delete from estoque where cadpro = :produto;
# delete from icadreq where requi = :produto and requi = '000000/25';
# delete from cadest where cadpro = :produto and grupo = '000';
# end
# end
# """)